In [1]:
import qiniu
import time
import requests

初始化，获取签名授权

In [2]:
access_key = "x34GCNDuYpCFsNtqkrri3Eagr6OANYyboWRZtWQv"
secret_key = "f_lfzdkBoEfOa_7a_0yrMOcIdfr-cHv9H463B5Xh"
# access_key = "MUpLpPtmVeiP7UWWteuSRBm5cHNeqSMndp4zh339"
# secret_key = "0ix4DVQZvPdlQNDIn0qTudaBj8h27FK9qWThMd5L"

# 构建鉴权对象
q = qiniu.Auth(access_key, secret_key)

数据或文件上传

In [7]:
# data upload
# key = 'pupy dog here.mp4'
# data = 'hello qiniu!'
# token = q.upload_token("yy-short-video")
# ret, info = qiniu.put_data(token, key, data)

# file upload
token = q.upload_token("yy-short-video")
key = 'user2/pupy dog here.mp4'
file_path = './data/upload/1185308075.mp4'
ret, info = qiniu.put_file(token, key, file_path, metadata=dict(name="tiantian"))
if ret is not None:
    print('All is OK', ret)
else:
    print(info)

All is OK {'hash': 'FkwkxLPsVhf1C3FK7EsSVprQLktO', 'key': 'user2/pupy dog here.mp4'}


资源管理

In [33]:
# 初始化 BucketManager 空间管理类
bucket = qiniu.BucketManager(q)

# 指定测试空间与文件名
bucket_name = "yy-short-video"
key = "user@1/video/5847849856.mp4"

ret, info = bucket.stat(bucket_name, key)
# ret, info = bucket.delete(bucket_name, key)
info

_ResponseInfo__response:<Response [200]>, exception:None, status_code:200, text_body:, req_id:A5UAAADdLNV9OJMX, x_log:-

In [90]:
prefix = None
# 列举条目
limit = 1
# 列举出除'/'的所有文件以及以'/'为分隔的所有前缀
delimiter = None
# 标记
marker = None
ret, eof, info = bucket.list(bucket_name, prefix, marker, limit, delimiter)
print(ret.get('items'))

[{'key': 'pdg.mp4', 'hash': 'FkwkxLPsVhf1C3FK7EsSVprQLktO', 'fsize': 1929256, 'mimeType': 'video/mp4', 'putTime': 16985474640459057, 'type': 0, 'status': 0, 'md5': '1954158dfcc00fa37bca49bbda9cfc2a', 'x-qn-meta': {'hhh': '123', 'nihao': 'aaa'}}]


In [106]:
bucket_domains = bucket.list_domains(bucket_name)
print(q.private_download_url(f"http://{bucket_domains[0][0]}/pdg.mp4"))
unix_time = ret.get('items')[0].get('putTime') // (10**7)
time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(unix_time))

'2023/10/29 10:44:24'

资源处理

In [12]:
# 是使用的队列名称,不设置代表不使用私有队列，使用公有队列。
pipeline = "small_eye"

# 要进行转码的转码操作。
fops = "vframe/jpg/offset/7/w/480/h/360"

saveas_key = qiniu.urlsafe_base64_encode(f"{bucket_name}:{key.split('.')[0]}_frame7.jpg")
fops = fops+"|saveas/"+saveas_key

pfop = qiniu.PersistentFop(q, bucket_name, pipeline)
ops = [fops]
ret, info = pfop.execute(key, ops, 1)

In [13]:
ret

{'persistentId': 'cn-east-2.01c201cwi9wdvuim3c00mvdajq000akz'}

In [4]:
url = "http://s31euirdk.bkt.clouddn.com/user%401/manifest/video/5847849856.mp4.json?e=1698758918&token=x34GCNDuYpCFsNtqkrri3Eagr6OANYyboWRZtWQv:_qNIXyubHGVZO_CHyqwkDzf43Yg="
data = requests.get(url).json()
data

{'username': 'y',
 'user_id': '1',
 'title': '狗狗真可爱',
 'submit_time': '12:11',
 'midea_type': 'video'}

In [6]:
import requests
import os
import json
import qiniu
import uuid
from datetime import datetime

from app_db.database import engine, get_db, SessionLocal
from app_db import schemas, crud, models
from app_login import get_password_hash
from app_utils.use_type import VideoCategoryType

In [2]:
root = r"D:\ZJU-use\Competition\七牛云\assets\short_videos"
with open(os.path.join(root, 'meta_data.json'), 'r') as f:
    data = json.load(f)
    print(data[0])

{'category': 'sport', 'title': '美女摔跤比赛#美女摔跤', 'author': '老张动物园', 'creat_time': 1698125783, 'collect_count': 2077, 'comment_count': 722, 'share_count': 15307, 'like_cnt': 11739, 'avatar_id': 'cb95729f2bbb4e92af61c7fd7a75d19f', 'cover_id': '9115c44157d54a12bddad431dfed3cbb', 'video_id': 'b0627f32eb3c4c7ea0d8dc583c84ea1c'}


In [3]:
access_key = "x34GCNDuYpCFsNtqkrri3Eagr6OANYyboWRZtWQv"
secret_key = "f_lfzdkBoEfOa_7a_0yrMOcIdfr-cHv9H463B5Xh"

q = qiniu.Auth(access_key, secret_key)
bucket = qiniu.BucketManager(q)
bucket_name = "yy-short-video"

In [4]:
def create_database_item():
    user = crud.get_user_by_username(db, username)
    if not user:
        hashed_password = get_password_hash("123")
        db_user = models.User(username=username, email=f"{username}@xx.com", is_active=True,
                              hashed_password=hashed_password, avatar_key=avatar_key)
        db.add(db_user)
        db.commit()
        db.refresh(db_user)
        user = db_user
    db_video = models.Video(title=title, author=username, owner_id=user.id,
                            category=VideoCategoryType(category),
                            video_key=video_key, avatar_key=avatar_key, cover_key=cover_key,
                            like_count=like_count, collect_count=collect_count,
                            comment_count=comment_count, share_count=share_count)
    db.add(db_video)
    db.commit()
    db.refresh(db_video)

In [7]:
token = q.upload_token(bucket_name)
db = SessionLocal()
for d in data:
    category = d.get("category")
    title = d.get("title")
    username = d.get("author")
    collect_count = d.get("collect_count")
    comment_count = d.get("comment_count")
    share_count = d.get("share_count")
    like_count = d.get("like_cnt")
    video_path = os.path.join(root, "video", "{}.mp4".format(d.get("video_id")))
    avatar_path = os.path.join(root, "pic", "avatar", "{}.png".format(d.get("avatar_id")))
    cover_path = os.path.join(root, "pic", "cover", "{}.png".format(d.get("cover_id")))

    video_key = f'video/{"{}.mp4".format(d.get("video_id"))}'
    avatar_key = f'avatar/{"{}.png".format(d.get("avatar_id"))}'
    cover_key = f'cover/{"{}.png".format(d.get("cover_id"))}'

    create_database_item()

    # ret1, info1 = qiniu.put_file(token, video_key, video_path)
    # ret2, info2 = qiniu.put_file(token, avatar_key, avatar_path)
    # ret3, info3 = qiniu.put_file(token, cover_key, cover_path)

db.close()

In [6]:
# if db.query(models.User).filter(models.User.username == username).first():
#     print("有")
# else:
#     print("无")
# db.query(models.User).filter(models.User.username == username).all()

t = "sport"
a: VideoCategoryType = VideoCategoryType(t)
a


<VideoCategoryType.SPORT: 'sport'>

In [11]:
db = SessionLocal()
# user = crud.get_users(db)[0]
# user.videos[0].video_url = "xxxx"
# print(user.videos[0].video_url)
print(db.query(models.Video).filter(models.Video.author == 'yi').all())
videos = db.query(models.Video).filter(models.Video.author == 'yi').all()
db.close()
# del db

In [33]:
datetime.now()

datetime.datetime(2023, 11, 4, 21, 33, 37, 404186)